# Limpieza de Netflix

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq

Leemos el archivo csv de Netflix anteriormente pedidos en "Datos_Netflix"

In [2]:
dfnetflix = pd.read_csv('data/peliculas_netflix.csv')

In [4]:
dfnetflix

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1789382,Steve,2025,tt32985279,1242404,movie,movie
1,1823218,The Woman in Cabin 10,2025,tt7130300,1290879,movie,movie
2,1996393,Swim to Me,2025,tt34682204,1484640,movie,movie
3,11042350,"My Father, the BTK Killer",2025,tt38466379,1550785,movie,movie
4,1780773,28 Years Later,2025,tt10548174,1100988,movie,movie
...,...,...,...,...,...,...,...
2899,1725885,The (Almost) Legends,2023,tt21737024,1148969,movie,movie
2900,1795372,Harta Tahta Raisa,2024,tt31495242,1250870,movie,movie
2901,1720462,Jurnal Risa by Risa Saraswati,2024,tt28448409,1141132,movie,movie
2902,1852883,Ahir Shah: Ends,2024,tt33246651,1332077,movie,movie


## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `'type'` y `'tmdb_type'` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [5]:
dfnetflix1 = dfnetflix
dfnetflix1['type'].unique()

array(['movie'], dtype=object)

In [6]:
dfnetflix1['tmdb_type'].unique()

array(['movie', 'tv'], dtype=object)

En la columna `'type'` si tiene un solo tipo, pero al parecer `'tmdb_type'` no, revisemos las filas con tipo distinto

In [7]:
dfnetflix1[dfnetflix1['tmdb_type'] == 'tv']

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
2766,3199048,The Rise of Wagner,2023,tt27480377,228220,tv,movie


Revisando el titulo 'The Rise of Wagner' en la pagina de imdb, este figura como un documental, al no estar separado en capitulos lo dejaremos como tipo `'movie'`. Al ya ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [8]:
dfnetflix2 = dfnetflix1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores nan

In [9]:
dfnetflix2['id'].isna().unique()

array([False])

In [10]:
dfnetflix2['title'].isna().unique()

array([False])

In [11]:
dfnetflix2['year'].isna().unique()

array([False])

In [12]:
dfnetflix2['imdb_id'].isna().unique()

array([False])

### Duplicados

In [13]:
dfnetflix2.duplicated().sum()

np.int64(0)

In [14]:
dfnetflix2[dfnetflix2.duplicated()]

,id,title,year,imdb_id,tmdb_id


In [15]:
dfnetflix3 = dfnetflix2.drop_duplicates(keep='first')
dfnetflix3

,id,title,year,imdb_id,tmdb_id
0,1789382,Steve,2025,tt32985279,1242404
1,1823218,The Woman in Cabin 10,2025,tt7130300,1290879
2,1996393,Swim to Me,2025,tt34682204,1484640
3,11042350,"My Father, the BTK Killer",2025,tt38466379,1550785
4,1780773,28 Years Later,2025,tt10548174,1100988
...,...,...,...,...,...
2899,1725885,The (Almost) Legends,2023,tt21737024,1148969
2900,1795372,Harta Tahta Raisa,2024,tt31495242,1250870
2901,1720462,Jurnal Risa by Risa Saraswati,2024,tt28448409,1141132
2902,1852883,Ahir Shah: Ends,2024,tt33246651,1332077


### Valores unicos

#### id
Deberia ser unico para cada pelicula

In [16]:
dfnetflix3['id'].value_counts()

id
1789382     1
1823218     1
1996393     1
11042350    1
1780773     1
           ..
1725885     1
1795372     1
1720462     1
1852883     1
1825357     1
Name: count, Length: 2904, dtype: int64

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [17]:
dfnetflix3['title'].value_counts()

title
The Call             3
Smile                3
Agneepath            2
Joy                  2
Cobweb               2
                    ..
6 Underground        1
Nonnas               1
The Pale Blue Eye    1
The Dig              1
Swim to Me           1
Name: count, Length: 2867, dtype: int64

revisemos porque se repiten los titulos de las peliculas

In [18]:
dfnetflix3[dfnetflix3['title'] == 'Monster']

,id,title,year,imdb_id,tmdb_id
1240,1259681,Monster,2021,tt2850272,489932
2049,1762146,Monster,2023,tt29941084,1203484


In [19]:
dfnetflix3[dfnetflix3['title'] == 'The Call']

,id,title,year,imdb_id,tmdb_id
264,1597901,The Call,2020,tt10530176,575604
270,1382254,The Call,2013,tt1911644,158011
1833,1617535,The Call,2020,tt12971924,746817


In [20]:
dfnetflix3[dfnetflix3['title'] == 'Smile']

,id,title,year,imdb_id,tmdb_id
12,1645477,Smile,2022,tt15474916,882598
2738,1698365,Smile,2022,tt15751968,1051335
2874,2749397,Smile,2023,tt22982202,1038522


In [21]:
dfnetflix3[dfnetflix3['title'] == 'Joy']

,id,title,year,imdb_id,tmdb_id
440,1749411,Joy,2024,tt10243672,1184495
2836,1487441,Joy,2018,tt8917752,539445


In [22]:
dfnetflix3[dfnetflix3['title'] == 'Cobweb']

,id,title,year,imdb_id,tmdb_id
664,1612542,Cobweb,2023,tt9100018,709631
1797,1697648,Cobweb,2023,tt21254598,901121


Como podemos ver por la diferencia de años y el 'imdb_id', puede que sean peliculas distintas con el mismo titulo, revisando la pagina de imdb se confirma esto

#### year

In [18]:
dfnetflix3['year'].value_counts()

year
2022    395
2024    339
2021    324
2023    311
2020    268
2019    226
2025    209
2018    198
2017    139
2016     85
2014     46
2015     45
2012     25
2013     24
2011     20
2001     19
2008     16
2006     15
2005     14
2004     14
2010     14
2007     13
2003     13
2009     12
1995     10
1997     10
1993      9
2002      8
2000      7
1998      7
1996      6
1999      6
1992      6
1994      5
1987      5
1991      4
1986      4
1988      4
1985      3
1982      3
0         2
1979      2
1989      2
1983      2
1984      2
1990      2
1978      2
1976      1
1975      1
1973      1
1971      1
1974      1
1968      1
1967      1
1977      1
1980      1
Name: count, dtype: int64

In [19]:
dfnetflix3['year'].value_counts().tail(10)

year
1978    2
1976    1
1975    1
1973    1
1971    1
1974    1
1968    1
1967    1
1977    1
1980    1
Name: count, dtype: int64

Nos llama la atencion una pelicula que salio en el año 0

In [20]:
dfnetflix3[dfnetflix3['year'] == 0]

,id,title,year,imdb_id,tmdb_id
2870,11039715,Rockstar: DUKI desde el fin del mundo,0,tt38203421,1546933
2890,11046064,Nouvelle École et après : Réussir ou mourir,0,tt38221275,1556160


Revisando en imdb estas peliculas salieron el 2025

In [21]:
dfnetflix3.loc[dfnetflix3['imdb_id'] == 'tt38203421', 'year'] = 2025
dfnetflix3.loc[dfnetflix3['imdb_id'] == 'tt38221275', 'year'] = 2025

In [22]:
dfnetflix3['year'].value_counts().tail(10)

year
1978    2
1976    1
1975    1
1973    1
1974    1
1971    1
1967    1
1968    1
1977    1
1980    1
Name: count, dtype: int64

#### imdb_id

In [23]:
dfnetflix3['imdb_id'].value_counts()

imdb_id
tt32985279    1
tt7130300     1
tt34682204    1
tt38466379    1
tt10548174    1
             ..
tt21737024    1
tt31495242    1
tt28448409    1
tt33246651    1
tt32441206    1
Name: count, Length: 2904, dtype: int64

#### Verificamos que el cambio de fecha se aplico correctamente

In [24]:
dfnetflix3[dfnetflix3['imdb_id'] == 'tt38203421']

,id,title,year,imdb_id,tmdb_id
2870,11039715,Rockstar: DUKI desde el fin del mundo,2025,tt38203421,1546933


In [25]:
dfnetflix3[dfnetflix3['imdb_id'] == 'tt38221275']

,id,title,year,imdb_id,tmdb_id
2890,11046064,Nouvelle École et après : Réussir ou mourir,2025,tt38221275,1556160


### Tipos de columnas

In [26]:
dfnetflix3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2904 entries, 0 to 2903
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2904 non-null   int64 
 1   title    2904 non-null   object
 2   year     2904 non-null   int64 
 3   imdb_id  2904 non-null   object
 4   tmdb_id  2904 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 113.6+ KB


Los tipos de las columnas estan bien

### Analisis de columna `year`

In [27]:
dfnetflix3['year'].describe()

count    2904.000000
mean     2018.906680
std         7.249638
min      1967.000000
25%      2018.000000
50%      2021.000000
75%      2023.000000
max      2025.000000
Name: year, dtype: float64

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [53]:

dfnetflix3.to_csv('api_netflix.csv', index=False, encoding='utf-8')
